In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np

from dataset import Dataset

In [ ]:
df = Dataset(csv_file_path=pathlib.Path('data', 'daylio_export_2024_05_16.csv'), remove=True)

In [ ]:
import math


ENTRIES = df.entries
OVERALL_AVERAGE = df.mood()
OVERALL_STD = df.std()
ACTIVITIES = {}
weights = []

for i, ca in enumerate(df.complete_analysis(n_threshold=0)):
    ACTIVITIES[ca.activity] = i
    weights.append(ca.mood_with_without.calc_change())

In [ ]:
def calc_loss(weights: list[float]) -> float:
    # without importance
    s = 0
    for entry in ENTRIES:
        s += (entry.mood - (
            OVERALL_AVERAGE * math.prod((1 + weights[ACTIVITIES[act]]) for act in entry.activities)
        ))**2
    return s / len(ENTRIES)

In [ ]:
def show_effects(weights: list[float]) -> None:
    for activity, weight in zip(ACTIVITIES, weights):
        print(f"{activity}: {weight:.2%}")


def show_bar(weights_: list[float]) -> None:
    fig, ax = plt.subplots(figsize=(5, 15))
    ax.barh(list(ACTIVITIES.keys())[::-1], weights_[::-1])
    plt.grid()
    plt.show()


def show_bars(weights1: list[float], weights2: list[float]) -> None:
    fig, ax = plt.subplots(figsize=(5, 15))
    ax.barh(list(ACTIVITIES.keys())[::-1], weights1[::-1], label="initial", alpha=0.7)
    ax.barh(list(ACTIVITIES.keys())[::-1], weights2[::-1], label="optimized", alpha=0.7)
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
# init loss
calc_loss(weights)

In [ ]:
show_bar(weights)

## `scipy` minimization

In [ ]:
import scipy.optimize as opt

In [ ]:
weights_np = np.array(weights)

res = opt.minimize(calc_loss, weights, method='BFGS', options={'disp': True})

In [ ]:
show_bars(weights, res.x)